# Milvus Lite
* https://milvus.io/docs/milvus_lite.md
* Python API: https://milvus.io/api-reference/pymilvus/v2.5.x/About.md     <-- 2.5.14

included in Python SDK, suitable for small vector search use cases.

WARN: 
- NOT working on Windows.
- with limits.

In [1]:
# !pip uninstall -y pymilvus

In [2]:
!pip install -U pymilvus

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [3]:
from pymilvus import MilvusClient
import numpy as np

# connect
client = MilvusClient("./milvus_demo.db")

In [4]:
# create collection
client.create_collection(
    collection_name="demo_collection",
    dimension=384  # The vectors we will use in this demo has 384 dimensions
)

In [5]:
# insert data
docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]

vectors = [[ np.random.uniform(-1, 1) for _ in range(384) ] for _ in range(len(docs)) ]
data = [ {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"} for i in range(len(vectors)) ]
res = client.insert(
    collection_name="demo_collection",
    data=data
)

In [6]:
import pandas as pd
import json

def pd_print(obj):
    if obj is None:
        return
    # from IPython.core.display import HTML
    # display(HTML(pd.json_normalize(obj).to_html()))
    from IPython.core.display import display_html, HTML
    display_html(HTML(pd.json_normalize(obj).to_html()))

In [7]:
# search
res = client.search(
    collection_name="demo_collection",
    data=[vectors[0]],
    filter="subject == 'history'",
    limit=2,
    output_fields=["text", "subject"],
)
print(res)

data: [[{'id': 0, 'distance': 0.9999999403953552, 'entity': {'text': 'Artificial intelligence was founded as an academic discipline in 1956.', 'subject': 'history'}}, {'id': 2, 'distance': -0.005298481322824955, 'entity': {'text': 'Born in Maida Vale, London, Turing was raised in southern England.', 'subject': 'history'}}]]


In [8]:
# query
res = client.query(
    collection_name="demo_collection",
    filter="subject == 'history'",
    output_fields=["text", "subject"],
)
print(res)

data: ["{'id': 0, 'text': 'Artificial intelligence was founded as an academic discipline in 1956.', 'subject': 'history'}", "{'id': 1, 'text': 'Alan Turing was the first person to conduct substantial research in AI.', 'subject': 'history'}", "{'id': 2, 'text': 'Born in Maida Vale, London, Turing was raised in southern England.', 'subject': 'history'}"], extra_info: {}


In [9]:
# delete
res = client.delete(
    collection_name="demo_collection",
    filter="subject == 'history'",
)
print(res)

[0, 1, 2]
